# Chapter 0: Compiling and Running Toy

Here is a simple program in the Toy programming language running in a RISC-V emulator, 
compiled using xDSL.
Try changing the program and observing the output:

In [1]:
from toy.compiler import compile, emulate_riscv

from xdsl.utils.exceptions import VerifyException

program = """
def main() {
  # Define a variable `a` with shape <2, 3>, initialized with the literal value.
  # The shape is inferred from the supplied literal.
  var a = [[1, 2, 3], [4, 5, 6]];

  # The literal tensor is implicitly reshaped: defining new
  # variables is the way to reshape tensors (element count must match).
  var b<3, 2> = [1, 2, 3, 4, 5, 6];

  # There is a built-in print instruction to display the contents of the tensor
  print(b);

  # Reshapes are implicit on assignment
  var c<2, 3> = b;

  # There are + and * operators for pointwise addition and multiplication
  var d = a + c;

  print(d);
}
"""

try:
    code = compile(program)
    emulate_riscv(code)
except VerifyException as e:
    print(e)

[[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
[[2.0, 4.0, 6.0], [8.0, 10.0, 12.0]]


It is also possible to use MLIR + LLVM to compile and execute Toy natively on your machine.
For this to work, MLIR and LLVM need to be built from source with the commit specified in the README, and the binaries used below must be in the path.

In [2]:
# Uncomment the following lines to execute
# !python -m toy examples/interpret.toy --emit=scf --ir \
# | xdsl-opt -p printf-to-llvm \
# | mlir-opt --test-lower-to-llvm \
# | mlir-cpu-runner --entry-point-result=void